[code of transformer from pytorch](https://github.com/pytorch/pytorch/blob/master/torch/nn/modules/transformer.py)



In [1]:
print('hello world')
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

hello world


In [2]:
from google.colab import drive
drive.mount('/content/dirve', force_remount=True)

Mounted at /content/dirve


In [3]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer

class TransformerModel(nn.Module):

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.decoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        decoder_layers = TransformerDecoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_decoder = TransformerDecoder(decoder_layers, nlayers)

        self.init_weights()

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        # self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, trg):
        trg_mask = model.generate_square_subsequent_mask(trg.size()[0]).to(device)
        # 分散表現に変換
        src = self.encoder(src)
        trg = self.decoder(trg)
        # 位置情報を入れる
        src = self.pos_encoder(src)
        trg = self.pos_encoder(trg)
        # モデルにデータを入れる
        output = self.transformer_encoder(src)
        # デコーダにエンコーダの出力を入れる（ここがおかしい）
        output = self.transformer_decoder(trg, output,tgt_mask = trg_mask)
        return output

In [4]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [5]:
!pip install janome

     |████████████████████████████████| 19.7MB 11.7MB/s 


In [6]:
import janome
from janome.tokenizer import Tokenizer
from torchtext import data
from torchtext import datasets
import random
import numpy as np

In [7]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [8]:
j_t = Tokenizer()
def tokenizer(text): 
    return [tok for tok in j_t.tokenize(text, wakati=True)]

In [9]:
# 重複のないデータセットか重複のあるデータセットを選ぶ
# flagがTrueの時重複のないデータを返す
def choose_dataset(flag = False):
  if flag:
    train, val, test = data.TabularDataset.splits(
        path="/content/dirve/My Drive/Colab Notebooks/data/", train='one_train.tsv',
        validation='one_val.tsv', test='one_test.tsv', format='tsv',
        fields=[('SRC', SRC), ('TRG', SRC)])
    filename = "/content/dirve/My Drive/Colab Notebooks/csv/one_result_transformer.csv"
  else:
    train, val, test = data.TabularDataset.splits(
        path="/content/dirve/My Drive/Colab Notebooks/data/", train='train.tsv',
        validation='val.tsv', test='test.tsv', format='tsv',
        fields=[('SRC', SRC), ('TRG', SRC)])
    filename = "/content/dirve/My Drive/Colab Notebooks/csv/result_transformer.csv"
  
  return train, val, test, filename

In [10]:
SRC = data.Field(sequential=True, 
                 tokenize=tokenizer,
                 init_token='<sos>',
                 eos_token='<eos>', 
                 lower=True)
train, val, test, filename = choose_dataset(True)

In [11]:
SRC.build_vocab(train, min_freq=1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_batch_size = 100
test_batch_size = 100
eval_batch_size = 100
train_iter, val_iter, test_iter = data.BucketIterator.splits((train, val, test), sort = False,  batch_sizes = (train_batch_size,eval_batch_size, test_batch_size), device= device)

In [12]:
ntokens = len(SRC.vocab.stoi) # the size of vocabulary
emsize = len(SRC.vocab.stoi) # embedding dimension
nhid = 512 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
dropout = 0.3 # the dropout value
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)

In [13]:
model

TransformerModel(
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.3, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=3576, out_features=3576, bias=True)
        )
        (linear1): Linear(in_features=3576, out_features=512, bias=True)
        (dropout): Dropout(p=0.3, inplace=False)
        (linear2): Linear(in_features=512, out_features=3576, bias=True)
        (norm1): LayerNorm((3576,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((3576,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.3, inplace=False)
        (dropout2): Dropout(p=0.3, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=3576, out_features=3576, bias=True)
        )
        (linear1): Linear(in_fea

In [14]:
criterion = nn.CrossEntropyLoss(ignore_index=SRC.vocab.stoi["<pad>"])
lr = 5 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

import time
def train(iterator):
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()
    for i, batch in enumerate(iterator):
        #print(i)
        src = batch.SRC
        trg = batch.TRG
        optimizer.zero_grad()
        output = model(src, trg)
        output = output[:].view(-1, output.shape[-1])
        trg = trg[:].view(-1)
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        

def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    with torch.no_grad():
      for i, batch in enumerate(data_source):
        data = batch.SRC
        targets = batch.TRG
        #src_mask = model.generate_square_subsequent_mask(data.shape[0]).to(device)
        output = eval_model(data, targets)
        output_flat = output[:].view(-1, output.shape[-1])
        targets = targets[:].view(-1)
        total_loss += len(data) * criterion(output_flat, targets).item()
    return total_loss / (len(data_source) - 1)

In [15]:
best_val_loss = float("inf")
epochs = 20 # The number of epochs
best_model = None
model.init_weights()

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(train_iter)
    val_loss = evaluate(model, val_iter)
    print('-' * 89)
    print('| epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          .format(epoch, (time.time() - epoch_start_time), val_loss))

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model

    scheduler.step()

-----------------------------------------------------------------------------------------
| epoch   1 | time: 105.66s | valid loss 250.16 | 
-----------------------------------------------------------------------------------------
| epoch   2 | time: 107.18s | valid loss 226.09 | 
-----------------------------------------------------------------------------------------
| epoch   3 | time: 105.48s | valid loss 188.38 | 
-----------------------------------------------------------------------------------------
| epoch   4 | time: 105.55s | valid loss 179.95 | 
-----------------------------------------------------------------------------------------
| epoch   5 | time: 106.39s | valid loss 163.43 | 
-----------------------------------------------------------------------------------------
| epoch   6 | time: 105.01s | valid loss 175.09 | 
-----------------------------------------------------------------------------------------
| epoch   7 | time: 106.13s | valid loss 171.70 | 
-------------

In [16]:
test_loss = evaluate(best_model, test_iter)
print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f}'.format(
    test_loss, math.exp(test_loss)))
print('=' * 89)

| End of training | test loss 150.13 | test ppl 158135333334821791427179906743753761436459132050814525796503257088.00


In [17]:
torch.save(best_model.state_dict(), "/content/dirve/My Drive/Colab Notebooks/model/transformer.pth")

In [18]:
model.state_dict(torch.load("/content/dirve/My Drive/Colab Notebooks/model/transformer.pth"))

OrderedDict([('pos_encoder.pe',
              tensor([[[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
                         0.0000e+00,  1.0000e+00]],
              
                      [[ 8.4147e-01,  5.4030e-01,  8.3868e-01,  ...,  1.0000e+00,
                         1.0052e-04,  1.0000e+00]],
              
                      [[ 9.0930e-01, -4.1615e-01,  9.1353e-01,  ...,  1.0000e+00,
                         2.0103e-04,  1.0000e+00]],
              
                      ...,
              
                      [[ 9.5625e-01, -2.9254e-01,  9.7021e-01,  ...,  8.7524e-01,
                         4.8143e-01,  8.7649e-01]],
              
                      [[ 2.7050e-01, -9.6272e-01,  7.3173e-01,  ...,  8.7519e-01,
                         4.8151e-01,  8.7644e-01]],
              
                      [[-6.6395e-01, -7.4778e-01, -1.7338e-01,  ...,  8.7514e-01,
                         4.8160e-01,  8.7639e-01]]], device='cuda:0')),
             ('transformer_

In [19]:
def gen_sentence(sentence, src_field, trg_field, model, batch_size):
  model.eval()
  in_str, out_str, pred, tmp = [], [], [], []
  length = len(sentence)

  with torch.no_grad():
    for _, batch in enumerate(sentence):
      src = batch.SRC
      trg = batch.TRG
      output = model(src, trg)
          
      for j in range(min(length, batch_size)):
        _, topi = output.data.topk(1)
        _, topi_s = output.data.topk(2) 
        for k in range(topi.size()[1]):
          if topi[:, k][0] == trg_field.vocab.stoi["<eos>"]:
            for m in range(topi_s.size()[0]):
              for l in range(topi_s.size()[1]):
                topi[m][l][0] = topi_s[m][l][1]
          for i in range(topi.size()[0]):
            if trg_field.vocab.itos[topi[:, k][i]] == "<eos>":
              break
            tmp.append(trg_field.vocab.itos[topi[:, k][i]])
          pred.append(tmp)
          tmp = []
        #print(src.size())
        in_str.append([src_field.vocab.itos[i.item()] for i in src[:,j] if src_field.vocab.itos[i.item()] != "<eos>"])
        out_str.append([trg_field.vocab.itos[i.item()] for i in trg[:,j] if trg_field.vocab.itos[i.item()] != "<eos>"])
      
  return in_str, out_str, pred

In [20]:
# 中間発表時にはテストデータは用いない
test_in, test_out, test_pred = [],[],[]
test_in, test_out, test_pred = gen_sentence(test_iter, SRC, SRC, best_model, test_batch_size)
val_in, val_out, val_pred = [],[],[]
val_in, val_out, val_pred = gen_sentence(val_iter, SRC, SRC, model, eval_batch_size)
train_in, train_out, train_pred = [],[],[]
train_in, train_out, train_pred = gen_sentence(train_iter, SRC, SRC, model, train_batch_size)

In [21]:
import pandas as pd

In [22]:
def convert_list_to_df(in_list, out_list, pred_list):
  row = []
  for i in range(len(in_list)):
    batch_input = in_list[i]
    batch_output = out_list[i]
    batch_pred = pred_list[i]
    input = [j for j in batch_input if j != "<pad>" and j != "<sos>" and j != "<eos>" and j != "<unk>"]
    output = [j for j in batch_output if j != "<pad>" and j != "<sos>" and j != "<eos>" and j != "<unk>"]
    predict = [j for j in batch_pred if j != "<pad>" and j != "<sos>" and j != "<eos>" and j != "<unk>"]
    input_str = "".join(input)
    output_str ="".join(output)
    predict_str = "".join(predict)
    row.append([input_str, output_str, predict_str])

  df = pd.DataFrame(row, columns=["input","answer","predict"])
  df = df.sort_values('input')
  return df

In [23]:
train_df = convert_list_to_df(train_in, train_out, train_pred)
val_df = convert_list_to_df(val_in, val_out, val_pred)
test_df = convert_list_to_df(test_in, test_out, test_pred)

In [24]:
df_s = pd.concat([train_df, test_df]).sort_values('input')

In [25]:
df_s.head(10)

,input,answer,predict
729,15分ぐらいまで,あっ,うーんそーー
84,15分前に事務所に,あっ,うーん
1049,18歳以上でないとあのやる,うーん,そそーー
1655,1センチ5ミリぐらいの,そうです,うーんそー
1147,20代だいぶ昔の話ですけれども20代の後半に仲良し3人組で女性ですがあのまー安い宿を使って軽...,あ,そーーーねー
1135,20代だいぶ昔の話ですけれども20代の後半に仲良し3人組で女性ですがあのまー安い宿を使って軽...,旅行,へえー
295,20代だいぶ昔の話ですけれども20代の後半に仲良し3人組で女性ですがあのまー安い宿を使って軽...,あ,そうはーーねーねーねーねー
1058,20年代戦後間もない,二十年代戦後ですね,へえーーーねーねーねー
2127,2つのおはじきを,はいうん二つの,うーんそーーねーねー
736,2年で辞めましてその後文化服装学院の大学の,あはい,へえー


In [26]:
df_s.to_csv(filename)